# Hands-On Time Series Workshop: *Identifying Arrhythmia Using Electrocardiogram Data*

In this workshop we will be using time series electrocardiogram (ECG) data to classify Arrhythmia. 

The workshop is split into the following parts: 
1. Load and explore the dataset
2. Pre-processing
3. Training and saving the model 
4. Deploying the model
5. Creating and adding a drift detector! 

The data pre-processing and model building are based upon the hardwork of Kaggle user Gregoire DC. You can see the original notebook [here](https://www.kaggle.com/gregoiredc/arrhythmia-on-ecg-classification-using-cnn). 

In [ ]:
# Colab has a load of packages pre-loaded into the environment. Installing the additional ones we require here.
!pip install seldon-deploy-sdk==1.4.1
!pip install alibi-detect==0.6.1
!pip install seaborn
!pip install dill

In [ ]:
# Data manipulation
import numpy as np
import pandas as pd

# Visualisation
import seaborn as sns
import matplotlib.pyplot as plt

# Scikit-learn for scoring models
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.utils import class_weight
from sklearn.utils import resample

# Keras acts as the model training library
from tensorflow import keras 
from tensorflow.keras.layers import Dense, Convolution1D, MaxPool1D, Flatten, Dropout
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import to_categorical

# Drift detection
from alibi_detect.cd import MMDDrift, KSDrift
from alibi_detect.cd.base import BaseUnivariateDrift
from alibi_detect.utils.saving import save_detector, load_detector

# Seldon Deploy SDK 
from seldon_deploy_sdk import Configuration, ApiClient, SeldonDeploymentsApi, DriftDetectorApi
from seldon_deploy_sdk.auth import OIDCAuthenticator

# For loading serialised objects
import dill 

# Utility library
import itertools

## Data Exploration

Download and unpack training and test data

In [ ]:
!wget https://storage.googleapis.com/seldon-koz-ecg/data/mitbih_test.csv.zip
!wget https://storage.googleapis.com/seldon-koz-ecg/data/mitbih_train.csv.zip
!unzip 'mitbih_test.csv.zip'
!unzip 'mitbih_train.csv.zip'

This workshop uses ECG data collected from Beth Israel Hospital Arrhythmia Laboratory between 1975 and 1979. The subjects were 25 men aged 32 to 89 years, and 22 women aged 23 to 89 years. Data was collected using a Holter monitor. You can find out more about the dataset [here](https://physionet.org/content/mitdb/1.0.0/).

**About the dataset**

* Number of Samples: 109446
* Number of Categories: 5
* Sampling Frequency: 125Hz
* Data Source: Physionet's MIT-BIH Arrhythmia Dataset
* Classes: ['N': 0, 'S': 1, 'V': 2, 'F': 3, 'Q': 4]

**Labels**

* N : Non-ecotic beats (normal beat) 
* S : Supraventricular ectopic beats 
* V : Ventricular ectopic beats 
* F : Fusion Beats 
* Q : Unknown Beats

More understanding around the labels: an ectopic beat is an extra beat which occurs prior to a normal beat. Fusion beats are a hybrid complex when a supraventricular (top section of your heart) and a ventricular (bottom section of the heart) impulse coincide. 

Next, we read the training and test data into Pandas.

In [ ]:
train_df=pd.read_csv('mitbih_train.csv',header=None)
test_df=pd.read_csv('mitbih_test.csv',header=None)

train_df.describe()

Each row is a single beat cycle, with each of the features a reading of the electrical activity across that cycle.

In [ ]:
train_df.iloc[0]

Converting the labels to integers, and counting each of their values. 

In [ ]:
train_df[187]=train_df[187].astype(int)
label_counts=train_df[187].value_counts()
print(label_counts)

Visualising the label imbalance with a doughnut chart. 

In [ ]:
plt.figure(figsize=(14,7))
my_circle=plt.Circle((0,0), 0.7, color='white')
plt.pie(label_counts, labels=['n','q','v','s','f'], colors=['red','green','blue','skyblue','orange'],autopct='%1.1f%%')
p=plt.gcf()
p.gca().add_artist(my_circle)
plt.show()

Clearly, there is a large imbalance in the labels with over 80% of them representing normal heartbeats. This imbalance needs to be rectified in order to create a model which can accurately pick out different types of arrhythmia. 

Therefore, we select 20,000 normal heartbeat samples. We then resample the remaining classes with replacement to have a resultant dataset with an equal proporition of each class. 

In [ ]:
random_state = 42

label_dfs = {}

for i in label_counts.index:
    label_dfs[i] = train_df[train_df[187]==i]

# The first class has 72k observations, so we sample down to 20000
label_dfs[0] = label_dfs[0].sample(n=20000, random_state=random_state)

# The remaining classes have less than 20000 observations and therefore we resample with replacement. 
for i in [1, 2, 3, 4]:
    label_dfs[i] = resample(label_dfs[i], replace=True, n_samples=20000, random_state=random_state)
    
rebalanced_df = pd.concat([i for i in label_dfs.values()])
rebalanced_label_counts = rebalanced_df[187].value_counts()
rebalanced_label_counts

Next we visualise a single instance of each class to see the difference in the signatures which they present.

To do so, we create a DataFrame with a single obeservation of each of the classes. 

In [ ]:
sample_df = rebalanced_df.groupby(187, group_keys=False).apply(lambda train_df: train_df.sample(n=1))
sample_df

In [ ]:
plt.figure(figsize=(20,12))

plt.plot(sample_df.iloc[0,:186], label="Cat. N")
plt.plot(sample_df.iloc[1,:186], label="Cat. S")
plt.plot(sample_df.iloc[2,:186], label="Cat. V")
plt.plot(sample_df.iloc[3,:186], label="Cat. F")
plt.plot(sample_df.iloc[4,:186], label="Cat. Q")

plt.legend()
plt.title("1-beat ECG for every category", fontsize=20)
plt.ylabel("Amplitude", fontsize=15)
plt.xlabel("Time (ms)", fontsize=15)

plt.show()

The final piece of pre-processing we do is to add Gaussian noise to our data. This will help us to generalise to unseen data better, by forcing our model to work harder to detect meaningful signal from random noise. 

In [ ]:
def add_gaussian_noise(signal):
    noise = np.random.normal(0, 0.3, 186)
    return (signal + noise)

single_sample = sample_df.iloc[0,:186]
noisy_sample=add_gaussian_noise(single_sample)

plt.subplot(2, 1, 1)
plt.plot(sample_df.iloc[0,:186])

plt.subplot(2, 1, 2)
plt.plot(noisy_sample)

plt.show()

Applying the Gaussian noise to the training data. 

In [ ]:
target_train=train_df[187]
target_test=test_df[187]

y_train=to_categorical(target_train)
y_test=to_categorical(target_test)

In [ ]:
X_train=train_df.iloc[:,:186].values
X_test=test_df.iloc[:,:186].values

for i in range(len(X_train)):
    X_train[i,:186]= add_gaussian_noise(X_train[i,:186])
    
X_train = X_train.reshape(len(X_train), X_train.shape[1],1)
X_test = X_test.reshape(len(X_test), X_test.shape[1],1)

## Model Training
For the sake of time we will not train our CNN live and use a pre-trained model. 

This pre-trained model was trained with the following architecture: 
```
# Input
in_shape = (X_train.shape[1],1)
inputs_cnn = Input(shape=(in_shape), name='inputs_cnn')

# 1st convolutional layer
conv1_1 = Convolution1D(64, (6), activation='relu', input_shape=in_shape)(inputs_cnn)
conv1_1 = BatchNormalization()(conv1_1)
pool1 = MaxPool1D(pool_size=(3), strides=(2), padding="same")(conv1_1)

# 2nd convolutional layer
conv2_1 = Convolution1D(64, (3), activation='relu', input_shape=in_shape)(pool1)
conv2_1 = BatchNormalization()(conv2_1)
pool2 = MaxPool1D(pool_size=(2), strides=(2), padding="same")(conv2_1)

# 3rd convolutional layer
conv3_1 = Convolution1D(64, (3), activation='relu', input_shape=in_shape)(pool2)
conv3_1 = BatchNormalization()(conv3_1)
pool3 = MaxPool1D(pool_size=(2), strides=(2), padding="same")(conv3_1)
flatten = Flatten()(pool3)

# Output
dense_end1 = Dense(64, activation='relu')(flatten)
dense_end2 = Dense(32, activation='relu')(dense_end1)
main_output = Dense(5, activation='softmax', name='main_output')(dense_end2)

```
The best model was saved using early stopping at epoch 25 of 40. We load back this pretrained model as well as the training history to explore. 

In [ ]:
!mkdir pretrained_model
!gsutil cp -r gs://tom-seldon-examples/time-series-workshop/1 pretrained_model
!gsutil cp gs://tom-seldon-examples/time-series-workshop/history.dill history.dill

In [ ]:
model = load_model("pretrained_model/1")

with open("history.dill", "rb") as file:
    history = dill.load(file)

We next define a function to evaluate our model training efforts. This plots the training and validation accuracy and losses of our model. 

In [ ]:
def evaluate_model(history, X_test, y_test, model):
    """
    Plots line graphs for the training/validation accuracy and losses. 
    """
    scores = model.evaluate((X_test), y_test, verbose=0)
    print("Accuracy: %.2f%%" % (scores[1]*100))
    
    fig1, ax_acc = plt.subplots()
    plt.plot(history['accuracy'])
    plt.plot(history['val_accuracy'])
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Model - Accuracy')
    plt.legend(['Training', 'Validation'], loc='lower right')
    plt.show()
    
    fig2, ax_loss = plt.subplots()
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Model- Loss')
    plt.legend(['Training', 'Validation'], loc='upper right')
    plt.plot(history['loss'])
    plt.plot(history['val_loss'])
    plt.show()
    target_names=['0','1','2','3','4']    

Evaluate the model and make predictions.

In [ ]:
evaluate_model(history, X_test, y_test, model)

Finally we plot a normalised confusion matrix to understand better how our model performs across the multiple classes. 

In [ ]:
def plot_confusion_matrix(cm, 
                          classes,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the normalised confusion matrix.
    """
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], '.2f'),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

y_pred=model.predict(X_test)
    
# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))
np.set_printoptions(precision=2)

# Plot confusion matrix
plt.figure(figsize=(10, 10))
plot_confusion_matrix(cnf_matrix, classes=['N', 'S', 'V', 'F', 'Q'],
                      title='Confusion matrix, with normalization')
plt.show()

### Deploying the Model
We now want to serve our model on the Seldon infrastructure.

For this we can use the pre-built Tensorflow container image, which allows us to rapidly deploy our newly trained model using TFServing. 

Firstly we save our model locally i.e. `model.save("pretrained_model")` 

We can then push it a cloud storage bucket ready for deployment. This has been done ahead of time and the model can be accessed at the following URI `gs://tom-seldon-examples/time-series-workshop/`

-----------

A couple of considerations: 
1. The TF pre-built image uses the SavedModel format, not Kera's older HDF5 format.
2. TFServing expects a subdirectory referring to the model version which then contains the model artifacts. You can create this manually when pushing to cloud storage e.g. `gs://tom-seldon-examples/time-series-workshop/1/saved_model.pb`

----------

We are now ready to deploy to Seldon. We will do this via the Python SDK provided to interact with Seldon Deploy. 

In [ ]:
SD_IP = "34.141.146.222"

config = Configuration()
config.host = f"http://{SD_IP}/seldon-deploy/api/v1alpha1"
config.oidc_client_id = "sd-api"
config.oidc_server = f"http://{SD_IP}/auth/realms/deploy-realm"
config.oidc_client_secret = "sd-api-secret"
config.auth_method = 'client_credentials'

def auth():
    auth = OIDCAuthenticator(config)
    config.id_token = auth.authenticate()
    api_client = ApiClient(configuration=config, authenticator=auth)
    return api_client

In [ ]:
YOUR_NAME = "YOUR-NAME" # Replace with your name
MODEL_NAME = "MODEL-NAME" # Replace with your model name

DEPLOYMENT_NAME = f"{YOUR_NAME}-{MODEL_NAME}"
MODEL_LOCATION = f"gs://tom-seldon-examples/time-series-workshop/"

In [ ]:
NAMESPACE = "seldon-gitops"

PREPACKAGED_SERVER = "TENSORFLOW_SERVER"

CPU_REQUESTS = "0.1"
MEMORY_REQUESTS = "1Gi"

CPU_LIMITS = "0.1"
MEMORY_LIMITS = "1Gi"

mldeployment = {
    "kind": "SeldonDeployment",
    "metadata": {
        "name": DEPLOYMENT_NAME,
        "namespace": NAMESPACE,
        "labels": {
            "fluentd": "true"
        }
    },
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "spec": {
        "name": DEPLOYMENT_NAME,
        "annotations": {
            "seldon.io/engine-seldon-log-messages-externally": "true"
        },
        "protocol": "seldon",
        "predictors": [
            {
                "componentSpecs": [
                    {
                        "spec": {
                            "containers": [
                                {
                                    "name": f"{DEPLOYMENT_NAME}-container",
                                    "resources": {
                                        "requests": {
                                            "cpu": CPU_REQUESTS,
                                            "memory": MEMORY_REQUESTS
                                        },
                                        "limits": {
                                            "cpu": CPU_LIMITS,
                                            "memory": MEMORY_LIMITS
                                        }
                                    }
                                }
                            ]
                        }
                    }
                ],
                "name": "default",
                "replicas": 1,
                "traffic": 100,
                "graph": {
                    "implementation": PREPACKAGED_SERVER,
                    "modelUri": MODEL_LOCATION,
                    "name": f"{DEPLOYMENT_NAME}-container",
                    "endpoint": {
                        "type": "REST"
                    },
                    "parameters": [],
                    "children": [],
                    "logger": {
                        "mode": "all"
                    }
                }
            }
        ]
    },
    "status": {}
}

In [ ]:
deployment_api = SeldonDeploymentsApi(auth())
deployment_api.create_seldon_deployment(namespace=NAMESPACE, mldeployment=mldeployment)

Our model should now be running as a fully fledged microservice. We can now log into Seldon Deploy:

* URL: http://34.141.146.222/seldon-deploy/
* Username: admin@seldon.io
* Password: 12341234

We can then test our new deployment using the following request: 
```
{
	"data": {
		"ndarray": [
			[
				[1.0],
				[0.76],
				[0.11],
				[0.0],
				[0.08],
				[0.08],
				[0.07],
				[0.05],
				[0.05],
				[0.04],
				[0.03],
				[0.03],
				[0.04],
				[0.03],
				[0.04],
				[0.04],
				[0.04],
				[0.05],
				[0.05],
				[0.05],
				[0.07],
				[0.07],
				[0.08],
				[0.1],
				[0.12],
				[0.13],
				[0.17],
				[0.2],
				[0.21],
				[0.24],
				[0.25],
				[0.26],
				[0.29],
				[0.27],
				[0.27],
				[0.24],
				[0.21],
				[0.17],
				[0.16],
				[0.12],
				[0.12],
				[0.11],
				[0.11],
				[0.1],
				[0.11],
				[0.1],
				[0.11],
				[0.1],
				[0.11],
				[0.11],
				[0.12],
				[0.11],
				[0.12],
				[0.11],
				[0.12],
				[0.11],
				[0.11],
				[0.11],
				[0.12],
				[0.11],
				[0.11],
				[0.1],
				[0.1],
				[0.09],
				[0.09],
				[0.08],
				[0.08],
				[0.08],
				[0.08],
				[0.07],
				[0.08],
				[0.07],
				[0.08],
				[0.07],
				[0.07],
				[0.07],
				[0.07],
				[0.07],
				[0.09],
				[0.1],
				[0.11],
				[0.11],
				[0.12],
				[0.12],
				[0.11],
				[0.1],
				[0.1],
				[0.1],
				[0.09],
				[0.07],
				[0.07],
				[0.05],
				[0.06],
				[0.05],
				[0.06],
				[0.05],
				[0.06],
				[0.05],
				[0.05],
				[0.0],
				[0.01],
				[0.19],
				[0.68],
				[0.98],
				[0.62],
				[0.04],
				[0.01],
				[0.09],
				[0.07],
				[0.07],
				[0.05],
				[0.04],
				[0.04],
				[0.03],
				[0.04],
				[0.04],
				[0.05],
				[0.04],
				[0.05],
				[0.04],
				[0.05],
				[0.05],
				[0.06],
				[0.07],
				[0.0],
				[0.0],
				[0.0],
				[0.0],
				[0.0],
				[0.0],
				[0.0],
				[0.0],
				[0.0],
				[0.0],
				[0.0],
				[0.0],
				[0.0],
				[0.0],
				[0.0],
				[0.0],
				[0.0],
				[0.0],
				[0.0],
				[0.0],
				[0.0],
				[0.0],
				[0.0],
				[0.0],
				[0.0],
				[0.0],
				[0.0],
				[0.0],
				[0.0],
				[0.0],
				[0.0],
				[0.0],
				[0.0],
				[0.0],
				[0.0],
				[0.0],
				[0.0],
				[0.0],
				[0.0],
				[0.0],
				[0.0],
				[0.0],
				[0.0],
				[0.0],
				[0.0],
				[0.0],
				[0.0],
				[0.0],
				[0.0],
				[0.0],
				[0.0],
				[0.0],
				[0.0],
				[0.0],
				[0.0],
				[0.0],
				[0.0],
				[0.0],
				[0.0],
				[0.0],
				[0.0],
				[0.0]
			]
		]
	}
}
```

## Drift Detection

Although powerful, modern machine learning models can be sensitive. Seemingly subtle changes in a data distribution can destroy the performance of otherwise state-of-the art models, which can be especially problematic when ML models are deployed in production. Typically, ML models are tested on held out data in order to estimate their future performance. Crucially, this assumes that the process underlying the input data `X` and output data `Y` remains constant.

Drift can be classified into the following types:
* **Covariate drift**: Also referred to as input drift, this occurs when the distribution of the input data has shifted `P(X) != Pref(X)`, whilst `P(Y|X) = Pref(Y|X)`. This may result in the model giving unreliable predictions.

* **Prior drift**: Also referred to as label drift, this occurs when the distribution of the outputs has shifted `P(Y) != Pref(Y)`, whilst `P(X|Y) = Pref(X|Y)`. This can affect the model’s decision boundary, as well as the model’s performance metrics.

* **Concept drift**: This occurs when the process generating `Y` from `X` has changed, such that `P(Y|X) != Pref(Y|X)`. It is possible that the model might no longer give a suitable approximation of the true process.

-----------------

In this instance we will train a Kolmgorov-Smirnov drift detector to pick up on covariate drift. The KS Drift detector applies a two-sample KS test to compare the distance between the new probability distribution and the reference distribution. 

This is done on a feature by feature basis and the results are then aggregated using a correction, i.e. Bonferroni, to determine whether drift has occurred overall within the sample. 

We will use the training set as our reference distribution. Creating our drift detector is then as simple as writing a single line of code:

In [ ]:
dd = KSDrift(X_train, p_val=.05, correction='bonferroni')

We can then easily get a prediction against a given batch of data and determine whether drift has occurred or not!

In [ ]:
preds = dd.predict(X_train)
labels = ["No", "Yes"]

print(f'Drift? {labels[preds["data"]["is_drift"]]}')

As expected, when using our training data as the new distribution we do not detect any drift. 

However, if we passed the instances of solely the ventricular ecoptic beats (class 2) we can see drift is now detected. 

In [ ]:
new_preds = dd.predict(label_dfs[2].values)

print(f'Drift? {labels[new_preds["data"]["is_drift"]]}')

Inspecting the `new_preds` object we can see that it contains the `is_drift` binary flag indicating whether drift has been detected after applying the Bonferroni correction to the feature wise distance.

In [ ]:
new_preds

We can easily save the artifacts related to our drift detector using Alibi Detect's `save_detector` utility function. 

In [ ]:
save_detector(dd, "drift_detector")

We then push this to our GCS bucket ready for deployment alongside our model. 

Remember to replace `<YOUR NAME>` with the same name as before! 

In [ ]:
!gsutil cp -r drift_detector gs://tom-seldon-examples/time-series-workshop/<YOUR NAME>/

We can then use the Seldon Deploy SDK to configure and deploy our shiny new drift detector!

In [ ]:
# TO DO: Add code to deploy the drift detector!
DD_URI = f'gs://tom-seldon-examples/time-series-workshop/{YOUR_NAME}/drift_detector/'
DD_NAME = "ks-drift"

dd_config = {'config': {'basic': 
                        {'drift_batch_size': '5',
                         'storage_uri': DD_URI},
                        'deployment': {'protocol': 'seldon.http'}
                        },
             'deployment_name': DEPLOYMENT_NAME,
             'detector_type': 'drift',
             'name': DD_NAME,
             'namespace': NAMESPACE
            }

In [ ]:
dd_api = DriftDetectorApi(auth())

dd_api.create_drift_detector_seldon_deployment(name=DEPLOYMENT_NAME, 
                                               namespace=NAMESPACE, 
                                               detector_data=dd_config)